In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvidia-smi

Sat Dec 10 08:47:44 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Load Data

In [ ]:
import pandas as pd
df_typo = pd.read_csv("/content/drive/MyDrive/NLP PROJECT/TODA(CLEANED)/BERT/TODA_CLEANED_01.csv").iloc[:,1:]
df_typo = df_typo.dropna()
X_cleaned = df_typo['text_cleaned']
# X_typo = df_typo['typo_corrected']
# df_typo = df_typo[~((X_typo.duplicated())|X_cleaned.duplicated())]
# X_cleaned = df_typo['text_cleaned']
# X_typo = df_typo['typo_corrected']
y = df_typo['class']
print(df_typo.shape)
df_typo = df_typo.reset_index(drop=True)
df_typo

(24783, 6)


,text_raw,text_cleaned,tokens_raw,tokens_stop,tokens_stems,class
0,934 8616\ni got a missed call from yo bitch,934 8616i got a missed call from yo bitch,"['934', '8616i', 'got', 'a', 'missed', 'call',...","['934', '8616i', 'got', 'missed', 'yo', 'bitch']","['934', '8616i', 'got', 'miss', 'yo', 'bitch']",1
1,RT @KINGTUNCHI_: Fucking with a bad bitch you ...,rt fucking with a bad bitch you gone need som...,"['rt', ' ', 'fucking', 'with', 'a', 'bad', 'bi...","['rt', 'fucking', 'bad', 'bitch', 'gone', 'nee...","['rt', 'fuck', 'bad', 'bitch', 'gone', 'need',...",1
2,RT @eanahS__: @1inkkofrosess lol my credit ain...,rt lol my credit ain't no where near good b...,"['rt', ' ', 'lol', 'my', 'credit', ""ain't"", '...","['rt', 'lol', 'credit', 'ai', 'near', 'good', ...","['rt', 'lol', 'credit', 'ai', 'near', 'good', ...",0
3,RT @Maxin_Betha Wipe the cum out of them faggo...,rt wipe the cum out of them faggot rt contac...,"['rt', ' ', 'wipe', 'the', 'cum', 'out', 'of',...","['rt', 'wipe', 'cum', 'faggot', 'rt', 'contact...","['rt', 'wipe', 'cum', 'faggot', 'rt', 'contact...",1
4,Niggas cheat on they bitch and don't expect no...,niggas cheat on they bitch and don't expect no...,"['niggas', 'cheat', 'on', 'they', 'bitch', 'an...","['niggas', 'cheat', 'bitch', 'expect', 'pay', ...","['nigga', 'cheat', 'bitch', 'expect', 'pay', '...",1
...,...,...,...,...,...,...
24778,The last at-bat at Yankee Stadium. Thanks for ...,the last atbat at yankee stadium thanks for ma...,"['the', 'last', 'atbat', 'at', 'yankee', 'stad...","['atbat', 'yankee', 'stadium', 'thanks', 'maki...","['atbat', 'yanke', 'stadium', 'thank', 'make',...",0
24779,@_bradleey LMFAOOOO yooo I lost my elevator pa...,lmfaoooo yooo i lost my elevator pass 1285571...,"[' ', 'lmfaoooo', 'yooo', 'i', 'lost', 'my', '...","['lmfaoooo', 'yooo', 'lost', 'elevator', 'pass...","['lmfaoooo', 'yooo', 'lost', 'elev', 'pass', '...",1
24780,"#porn,#android,#iphone,#ipad,#sex,#xxx, | #Ana...",pornandroidiphoneipadsexxxx anal hardcore br...,"['pornandroidiphoneipadsexxxx', ' ', 'anal', '...","['pornandroidiphoneipadsexxxx', 'anal', 'hardc...","['pornandroidiphoneipadsexxxx', 'anal', 'hardc...",1
24781,RT @JennyJohnsonHi5: Just when I thought Justi...,rt just when i thought justin bieber couldn't...,"['rt', ' ', 'just', 'when', 'i', 'thought', 'j...","['rt', 'thought', 'justin', 'bieber', 'anymore...","['rt', 'thought', 'justin', 'bieber', 'anymor'...",1


## K_FOLD

In [ ]:
%%time
import ast
from sklearn.model_selection import KFold
# K_Fold Cross Validation

df_typo['tokens_raw'] = df_typo['tokens_raw'].apply(lambda x: ast.literal_eval(str(x)))
kf = KFold(n_splits=3)
K_FOLD_LIST = []
k=1
for train_index, test_index in kf.split(df_typo):
  K_FOLD_LIST.append([train_index, test_index,k,df_typo['tokens_raw'].iloc[test_index].apply(len).sum()])
  k+=1

CPU times: user 1.17 s, sys: 163 ms, total: 1.33 s
Wall time: 1.67 s


## Dataset_list

In [ ]:
dataset_list = [X_cleaned]
dataset_name = ["Clean"]

dataset = {na:li for na,li in zip(dataset_name,dataset_list)}

# Import Models

In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 22.3 MB/s 
     |████████████████████████████████| 182 kB 73.1 MB/s 
     |████████████████████████████████| 7.6 MB 67.8 MB/s 


In [ ]:
Model_list = []

## ALBERT

12 repeating layers

128 embedding dimension

768 hidden dimension

12 attention heads

11M parameters

In [ ]:
from transformers import AlbertTokenizerFast, TFAlbertForSequenceClassification
tokenizer_albert = AlbertTokenizerFast.from_pretrained('albert-base-v2')
model_albert = TFAlbertForSequenceClassification.from_pretrained('albert-base-v2')
Model_list.append((AlbertTokenizerFast,TFAlbertForSequenceClassification,'ALBERT','albert-base-v2','TF'))

All model checkpoint layers were used when initializing TFAlbertForSequenceClassification.

Some layers of TFAlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## RoBERTa

In [ ]:
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification
tokenizer_roberta = RobertaTokenizer.from_pretrained('roberta-base')
model_roberta = TFRobertaForSequenceClassification.from_pretrained('roberta-base')
Model_list.append((RobertaTokenizer,TFRobertaForSequenceClassification,'RoBERTa','roberta-base','TF'))

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## DistilBERT

In [ ]:
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
tokenizer_distilbert = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model_distilbert = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
Model_list.append((DistilBertTokenizer,TFDistilBertForSequenceClassification,'DistilBERT','distilbert-base-uncased','TF'))

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_projector', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'classifier', 'dropout_872']
You should probably TRAIN this model on a down-stream task to be able to use 

## ELECTRA

In [ ]:
from transformers import ElectraTokenizer, TFElectraForSequenceClassification
tokenizer_electra = ElectraTokenizer.from_pretrained('google/electra-small-discriminator')
model_electra = TFElectraForSequenceClassification.from_pretrained('google/electra-small-discriminator')
Model_list.append((ElectraTokenizer,TFElectraForSequenceClassification,'ELECTRA','google/electra-small-discriminator','TF'))

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/54.5M [00:00<?, ?B/s]

Some layers from the model checkpoint at google/electra-small-discriminator were not used when initializing TFElectraForSequenceClassification: ['discriminator_predictions']
- This IS expected if you are initializing TFElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## MobileBERT

In [ ]:
from transformers import MobileBertTokenizer, TFMobileBertForSequenceClassification
tokenizer_mobilebert = MobileBertTokenizer.from_pretrained('google/mobilebert-uncased')
model_mobilebert = TFMobileBertForSequenceClassification.from_pretrained('google/mobilebert-uncased')
Model_list.append((MobileBertTokenizer,TFMobileBertForSequenceClassification,'MobileBERT','google/mobilebert-uncased','TF'))

All model checkpoint layers were used when initializing TFMobileBertForSequenceClassification.

Some layers of TFMobileBertForSequenceClassification were not initialized from the model checkpoint at google/mobilebert-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# TRAINING & EVALUATION

In [ ]:
for i in range(len(Model_list)):
  print(f"{i+1} - Tokenizer[0] : {str(Model_list[i][0]).split('.')[-1][:-2]}",end='\t\t')
  print(f"Model[1] : {str(Model_list[i][1]).split('.')[-1][:-2]:38s}",end='\t')
  print(f"Name[2] : {Model_list[i][2]:10s}",end='\t')
  print(f"Pretrained[3] : {Model_list[i][3]}")

1 - Tokenizer[0] : ElectraTokenizer		Model[1] : TFElectraForSequenceClassification    	Name[2] : ELECTRA   	Pretrained[3] : google/electra-small-discriminator


In [ ]:
import time
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

In [ ]:
from tensorflow.python import train
eval = pd.DataFrame([[np.nan for i in range(11)]])
eval.columns = ['Model','Dataset','K_fold','Train_Score(ACC)','Train_Score(ROC_AUC)','Train_Score(F1)','Test_Score(ACC)','Test_Score(ROC_AUC)','Test_Score(F1)','Inference_Time','words_nums']
eval = eval.iloc[1:]
 
 
for data_name in dataset_name:
  for tokenizer, model_seq, model_name, pretrained, plat in Model_list:
    print(f"Tokenizer[0] : {str(tokenizer).split('.')[-1][:-2]}",end='\t\t')
    print(f"Model[1] : {str(model_seq).split('.')[-1][:-2]:38s}",end='\t')
    print(f"Name[2] : {model_name:10s}",end='\t')
    print(f"Pretrained[3] : {pretrained}")
 
    # Performance Base DataFrame
    LR_list_base = [model_name] 
    LR_list_base.append(data_name) # CLEAN
 
    # Load Data
    X = dataset[data_name]
 
    # K_Fold Cross Validation
    for train_index, test_index, k, word_nums in K_FOLD_LIST:
      tokenizer = tokenizer.from_pretrained(pretrained)
      LR = model_seq.from_pretrained(pretrained)
      optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
      LR.compile(optimizer=optimizer, loss=tf.keras.losses.BinaryCrossentropy())
 
 
      # Get Train, Test
      X_train, X_test = X.iloc[train_index].to_list(), X.iloc[test_index].to_list()
      y_train, y_test = y.iloc[train_index].to_list(), y.iloc[test_index].to_list()
 
      encoding_time = time.time()
      train_encodings = tokenizer(X_train, truncation=True, padding=True)
      test_encodings = tokenizer(X_test, truncation=True, padding=True)
    
      print("encoding time : ", time.time()-encoding_time)
 
      train_dataset = 0
      test_dataset = 0
 
      dataset_time = time.time()
      train_dataset = tf.data.Dataset.from_tensor_slices((
          dict(train_encodings),
          y_train
      ))
      test_dataset = tf.data.Dataset.from_tensor_slices((
          dict(test_encodings),
          y_test
      ))


      print("dataset time : ", time.time()-dataset_time)
 
      # Fit and Inference

      LR.fit(train_dataset.shuffle(1000).batch(16),epochs=3)
 
      print(f"{data_name} - {k} Trained")
      fitted = LR.predict(train_dataset.shuffle(1000).batch(16))
      fitted = tf.nn.softmax(fitted.logits, axis=1)
      fitted_proba = fitted[:,1]
      fitted = [np.argmax(res) for res in fitted]


      start = time.time()
      pred = LR.predict(test_dataset.shuffle(1000).batch(16))
      pred = tf.nn.softmax(pred.logits, axis=1)
      inference_time = time.time()-start
      pred_proba = pred[:,1]
      pred = [np.argmax(res) for res in pred_proba]

      
      print(f"{data_name} - {k} Inferenced : {inference_time}s",end='\t')
 
      # Evaluate
      train_acc = accuracy_score(y_train,fitted)
      train_auc = roc_auc_score(y_train,fitted_proba)
      train_f1 = f1_score(y_train,fitted)
      test_acc = accuracy_score(y_test,pred)
      test_auc = roc_auc_score(y_test,pred_proba)
      test_f1 = f1_score(y_test,pred)
      print(f"train ACC : {train_acc} test ACC : {test_acc} test F1 : {test_f1}")
 
      LR_list = LR_list_base.copy()
      LR_list.append(k)
      LR_list.append(train_acc)
      LR_list.append(train_auc)
      LR_list.append(train_f1)
      LR_list.append(test_acc)
      LR_list.append(test_auc)
      LR_list.append(test_f1)
      LR_list.append(inference_time)
      LR_list.append(word_nums)
 
      # eval = eval.append(pd.DataFrame([LR_list],columns=eval.columns))
      # eval.to_csv(f"/content/drive/MyDrive/NLP PROJECT/PERFORMANCE/{model_name}_performance_{k}.csv")
      # eval = pd.DataFrame([[np.nan for i in range(11)]])
      # eval.columns = ['Model','Dataset','K_fold','Train_Score(ACC)','Train_Score(ROC_AUC)','Train_Score(F1)','Test_Score(ACC)','Test_Score(ROC_AUC)','Test_Score(F1)','Inference_Time','words_nums']
      # eval = eval.iloc[1:]

Tokenizer[0] : ElectraTokenizer		Model[1] : TFElectraForSequenceClassification    	Name[2] : ELECTRA   	Pretrained[3] : google/electra-small-discriminator


Some layers from the model checkpoint at google/electra-small-discriminator were not used when initializing TFElectraForSequenceClassification: ['discriminator_predictions']
- This IS expected if you are initializing TFElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


encoding time :  20.6520094871521
dataset time :  19.629234075546265
Epoch 1/3
1033/1033 [==============================] - 143s 117ms/step - loss: 0.3032
Epoch 2/3
1033/1033 [==============================] - 120s 116ms/step - loss: 0.1838
Epoch 3/3
1033/1033 [==============================] - 119s 116ms/step - loss: 0.1767
Clean - 1 Trained
517/517 [==============================] - 29s 51ms/step
Clean - 1 Inferenced : 28.943520307540894s	train ACC : 0.20947827139571482 test ACC : 0.16838155187023363 test F1 : 0.0


Some layers from the model checkpoint at google/electra-small-discriminator were not used when initializing TFElectraForSequenceClassification: ['discriminator_predictions']
- This IS expected if you are initializing TFElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


encoding time :  9.432066679000854
dataset time :  33.40306854248047
Epoch 1/3
1033/1033 [==============================] - 139s 119ms/step - loss: 0.3482
Epoch 2/3
1033/1033 [==============================] - 122s 118ms/step - loss: 0.1611
Epoch 3/3
1033/1033 [==============================] - 119s 115ms/step - loss: 0.1801
Clean - 2 Trained
517/517 [==============================] - 28s 50ms/step
Clean - 2 Inferenced : 28.572771072387695s	train ACC : 0.2750877617721825 test ACC : 0.16753419682847112 test F1 : 0.0


Some layers from the model checkpoint at google/electra-small-discriminator were not used when initializing TFElectraForSequenceClassification: ['discriminator_predictions']
- This IS expected if you are initializing TFElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


encoding time :  9.303468704223633
dataset time :  16.997977256774902
Epoch 1/3
1033/1033 [==============================] - 136s 116ms/step - loss: 0.3665
Epoch 2/3
1033/1033 [==============================] - 120s 116ms/step - loss: 0.1619
Epoch 3/3
1033/1033 [==============================] - 120s 116ms/step - loss: 0.1509
Clean - 3 Trained
517/517 [==============================] - 23s 39ms/step
Clean - 3 Inferenced : 23.52329444885254s	train ACC : 0.8320421256506476 test ACC : 0.16801839970947827 test F1 : 0.0


# SIMPLE TEST

In [ ]:
inputs = "fuck niggas cheat on they bitch"
inputs = tokenizer(inputs, truncation=True, padding=True, return_tensors="tf")
a=tf.nn.softmax(model_electra(inputs)[0].numpy(), axis=1).numpy()[0]
if(a[0]<a[1]):
  print("Hate Speech Detected")
else:
  print("Not Hate Speech")

Hate Speech Detected


In [ ]:
inputs = "Hello, Have a good day"
inputs = tokenizer(inputs, truncation=True, padding=True, return_tensors="tf")
tf.nn.softmax(model_electra(inputs)[0].numpy(), axis=1).numpy()[0]
if(a[0]<a[1]):
  print("Hate Speech Detected")
else:
  print("Not Hate Speech")

Not Hate Speech


In [ ]:
inputs = "I thi like  fe fewp nk it's so fucking bueatiful voice. I cunta love this is love music so much. It's my favorite."
inputs = tokenizer(inputs, truncation=True, padding=True, return_tensors="tf")
tf.nn.softmax(model_electra(inputs)[0].numpy(), axis=1).numpy()[0]

array([0.49664566, 0.5033543 ], dtype=float32)